In [3]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pickle as pkl

##### 加载预存数据并构建KNN分类器

In [2]:
train_previous_increase_ratio = np.load("../data/interim/stock500_2023_2024_previous_increase_ratio.npz")["previous_increase_ratio"]
train_post_increase_ratio = np.load("../data/interim/stock500_2023_2024_post_increase_ratio.npz")["post_increase_ratio"]
# 构建最近邻模型
nbrs = NearestNeighbors(n_neighbors=50, metric='cosine')
nbrs.fit(train_previous_increase_ratio)

# 查询最近邻
distances, nearest_indices = nbrs.kneighbors(train_previous_increase_ratio)
print("nearest neighbors computed")

NameError: name 'np' is not defined

### 根据增长率的余弦相似度构造正负样本

In [15]:
sampled_candidates = []
positive_sample_count = 5
total_sample_count = 20
for i, indices in enumerate(nearest_indices):
    positive_candidate_index = np.random.choice(indices, size=positive_sample_count+1, replace=False)[1:]
    negative_candidate_index = np.random.choice(np.arange(train_previous_increase_ratio.shape[0]), size=total_sample_count-positive_sample_count, replace=False)
    candidate_index = np.concatenate([positive_candidate_index, negative_candidate_index])
    candidate_distance = cosine_similarity((train_post_increase_ratio[i]-1).reshape((1, -1)), train_post_increase_ratio[candidate_index]-1).flatten()
    sampled_candidates.append((i, candidate_index, candidate_distance))
    if i % 1000 == 0:
        print(i, end=",")
pkl.dump(sampled_candidates, open("../data/processed/stock500_2023_2024_sampled_candidates_v2.pkl", 'wb'))

0,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000,16000,17000,18000,19000,20000,21000,22000,23000,24000,25000,26000,27000,28000,29000,30000,31000,32000,33000,34000,35000,36000,37000,38000,39000,40000,41000,42000,43000,44000,45000,46000,47000,48000,49000,50000,51000,52000,53000,54000,55000,56000,57000,58000,59000,60000,61000,62000,63000,64000,65000,66000,67000,68000,69000,70000,71000,72000,73000,74000,75000,76000,77000,78000,79000,80000,81000,82000,83000,84000,85000,86000,87000,88000,89000,90000,91000,92000,93000,94000,95000,96000,97000,98000,99000,100000,101000,102000,103000,104000,105000,106000,107000,108000,109000,110000,111000,112000,113000,114000,115000,116000,117000,118000,119000,120000,121000,122000,123000,124000,125000,126000,127000,128000,129000,130000,131000,132000,133000,134000,135000,136000,137000,138000,139000,140000,141000,142000,143000,144000,145000,146000,147000,148000,149000,150000,151000,152000,153000,154000,155000,156000,157000,158000,

##### 验证数据是否正确

In [1]:
print("Shape of train_previous_increase_ratio: ", train_previous_increase_ratio.shape)
print("Shape of train_post_increase_ratio: ", train_post_increase_ratio.shape)
# 随机选择样本中的一个
selected_sample_index = np.random.choice(np.arange(len(sampled_candidates)), 1, replace=False)[0]
idx, candidate_idxs, candidate_distances = sampled_candidates[selected_sample_index]
for i in range(len(candidate_idxs)):
    idx2 = candidate_idxs[i]
    distance = candidate_distances[i]
    distance2 = cosine_similarity(train_post_increase_ratio[idx].reshape(1, -1)-1, train_post_increase_ratio[idx2].reshape(1, -1)-1)[0][0]
    assert abs(distance-distance2) < 1-6

NameError: name 'train_previous_increase_ratio' is not defined

In [5]:
sampled_candidates = pkl.load(
    open("../data/processed/stock500_2023_2024_sampled_candidates_v2.pkl", "rb")
)
train_previous_increase_ratio = np.load("../data/interim/stock500_2023_2024_previous_increase_ratio.npz")["previous_increase_ratio"]
train_post_increase_ratio = np.load("../data/interim/stock500_2023_2024_post_increase_ratio.npz")["post_increase_ratio"]
valid_train_samples = (np.min(train_previous_increase_ratio, axis=1) > 0) & (np.min(train_post_increase_ratio, axis=1) > 0)
sampled_candidates_clean = []
for idx, candidate_idxs, candidate_distances in sampled_candidates:
    if valid_train_samples[idx]:
        sampled_candidates_clean.append((idx, candidate_idxs, candidate_distances))
pkl.dump(sampled_candidates_clean, open("../data/processed/stock500_2023_2024_sampled_candidates_v2_clean.pkl", 'wb'))